In [1]:
import functools as fn
import datetime
import math

import pandas as pd
import numpy as np
import scipy.stats as scs

# from sklearnex import patch_sklearn ## Must run patch before importing other sklearn functions!!!
# patch_sklearn()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

from matplotlib import pyplot as plt

In [2]:
t = pd.read_csv('./query_trades_AAPL_20200106_born202210121745.csv',)
q = pd.read_csv('./query_quotes_AAPL_20200106_born202210121744.csv',)

c:\Users\E\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
from taq_data_cleaning import gen_targets_temporal, gen_targets_events, gen_basic_features_TAQ

In [4]:
gen_basic_features_TAQ?

Signature:
gen_basic_features_TAQ(
    t_v,
    q_v,
    day=datetime.datetime(2020, 1, 6, 0, 0),
    str_filename='clean_features_',
)
Docstring:
From clean trades and quotes dataframes built from TAQ data, generates simple features 

num_events: int
    number of events ahead to build predict targets
File:      c:\users\e\projects\ams520_taq_project\taq_data_cleaning.py
Type:      function


In [5]:
df_features = gen_basic_features_TAQ(t, q,)

In [5]:
df_features.head()

,index,time,ask_price,bid_price,ask_volume,bid_volume,mid_price,spread,dPask_dt,dPbid_dt,dVask_dt,dVbid_dt,avg_trade_price_10ms,avg_trade_price_100ms,avg_trade_price_1s,avg_trade_price_10s,avg_trade_volume_10ms,avg_trade_volume_100ms,avg_trade_volume_1s,avg_trade_volume_10s
id,,,,,,,,,,,,,,,,,,,,
2214126,42179,35100.044580,295.08,295.04,1.0,1.0,295.060,0.04,-0.01,-0.01,-2.0,-1.0,81.468781,81.468781,73.762734,92.485452,38.522388,38.522388,33.826923,23.284404
2214141,42181,35100.044763,295.07,295.04,1.0,1.0,295.055,0.03,0.00,0.00,-4.0,0.0,82.530920,82.530920,76.081708,92.538789,37.041958,37.041958,33.943396,22.974242
2214146,42182,35100.044834,295.06,295.04,2.0,1.0,295.050,0.02,0.00,0.01,-1.0,0.0,85.722849,85.722849,82.688227,93.314248,37.817568,37.817568,36.286624,23.287651
2214167,42183,35100.045857,295.06,295.04,3.0,1.0,295.050,0.02,-0.02,0.00,2.0,0.0,87.061311,87.061311,95.780210,95.865806,38.918033,38.918033,29.251309,22.156566
2214865,42184,35100.094862,295.06,295.04,2.0,1.0,295.050,0.02,-0.03,-0.03,0.0,0.0,84.716192,84.716192,95.503117,94.753454,44.059406,44.059406,34.942029,24.146264


In [6]:
df_events = gen_targets_events(q,num_events=30)

In [7]:

df_targets_time = gen_targets_temporal(q,prediction_interval=100)

In [8]:
df_targets_time.sample(8)

,index,id,time,midup100ms,middown100ms,mideq100ms,spreadup100ms,spreaddown100ms,spreadeq100ms
276741,276741,18824031,48062.054981,False,False,True,False,False,True
316616,316616,22037368,51644.690476,False,False,True,False,False,True
320518,320518,22328225,51953.879979,False,False,True,False,False,True
111320,111320,8288129,38595.086408,False,True,False,False,False,True
199560,199560,13781621,42841.662448,False,False,True,False,False,True
266866,266866,18129160,47281.507572,False,False,True,False,False,True
34952,34952,3783524,35894.606952,False,False,True,False,False,True
77054,77054,6363842,37355.809590,False,True,False,False,False,True


In [12]:
def create_df_combined(df_features, df_targets_time, df_events=None ):

    #Explicit merge, should use left join instead
    try:
        df_features.set_index('id', inplace=True)
    except:
        pass
    df_features.sort_index(inplace=True)
    try:
        df_targets_time.set_index('id', inplace=True) 
    except:
        pass
    df_targets_time.sort_index(inplace=True)
    ntarg = []
    for col in df_targets_time.columns:
        if 'mid' in col or 'spread' in col:
            ntarg += [col]
    ind_comb = set(df_features.index.to_numpy()).intersection(set(df_targets_time.index.to_numpy()))
    df_combine = df_features.loc[ind_comb]
    df_combine[ntarg] = df_targets_time[ntarg].loc[ind_comb]
    if ~isinstance(df_events, type(None)):
        try:
            df_events.set_index('id', inplace=True)
        except:
            pass
        nevent = []
        for col in df_events.columns:
            if 'mid' in col or 'spread' in col:
                nevent += [col]
        ind_comb = ind_comb.intersection(set(df_events.index.to_numpy()))
   
        df_combine[nevent] = df_events[nevent].loc[ind_comb]
    return df_combine
    
    

    

In [13]:
df_combine =  create_df_combined(df_features, df_targets_time, df_events)

In [14]:
df_combine.reset_index(inplace=True)
df_combine.to_feather("clean_combine_30e_50ms_AAPL_20200106.f") ## checkpoint combined features/targets

In [46]:
df_combine.head()

,level_0,id,index,time,ask_price,bid_price,ask_volume,bid_volume,mid_price,spread,...,mideq100ms,spreadup100ms,spreaddown100ms,spreadeq100ms,midup,middown,mideq,spreadup,spreaddown,spreadeq
0,0,3145728,60760,35567.630740,295.96,295.93,1.0,2.0,295.945,0.03,...,True,False,False,True,False,True,False,False,False,True
1,1,7340033,139161,37963.163817,297.15,297.14,2.0,2.0,297.145,0.01,...,True,False,False,True,False,True,False,False,False,True
2,2,3145730,60761,35567.630920,295.97,295.93,4.0,2.0,295.950,0.04,...,True,False,False,True,False,True,False,False,False,True
3,3,13631491,240204,42697.359376,298.45,298.43,1.0,1.0,298.440,0.02,...,True,False,False,True,False,True,False,False,False,True
4,4,3145732,60762,35567.630974,295.97,295.93,3.0,2.0,295.950,0.04,...,True,False,False,True,False,True,False,False,False,True


In [15]:
import plotly.graph_objects as go
cols = []
for col in df_combine.columns:
    if 'mid' in col or 'spread' in col:
        cols += [col]
cols = cols[2:]
fig = go.Figure()
fig.add_bar(x=cols,y=df_combine[cols].sum()/len(df_combine))
fig.update_layout(title_text="fraction of targets True")
fig.show()

In [10]:
rng = np.random.default_rng(3407)

In [16]:
#### use predict set {-1, 1}
predict = 'midup100ms'
split  = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=3407)
n =10000
ind_select =  rng.choice(np.arange(len(df_combine)),size=n)
#ind_select = np.arange(n) # no randomization
try:
    df_combine.reset_index(inplace=True)
except ValueError:
    pass # already rest index
df_select = df_combine.iloc[ind_select,:]
for train_index, test_index in split.split(df_select, df_select[predict]):
    train_set = df_combine[:n].loc[train_index]
    test_set = df_combine[:n].loc[test_index]
y_train = np.where(np.array(train_set[predict]), -1.0, 1.0)
svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=2, C=0.25, gamma=1.0, coef0=1.0, verbose=True)) # adjusting gamma for increased smoothness - trying to avoid trivial prediction
])
feature_names = ['ask_price', 'bid_price', 'ask_volume', 'bid_volume', 
                   'mid_price', 'spread', 
                   'dPask_dt', 'dPbid_dt', 'dVbid_dt', 'dVask_dt',
                   'avg_trade_price_10ms','avg_trade_volume_10ms',
                   'avg_trade_price_100ms','avg_trade_volume_100ms',
                   'avg_trade_price_1s','avg_trade_volume_1s',
                   'avg_trade_price_10s','avg_trade_volume_10s' 
                  ]
svm.fit(train_set[feature_names], 
        y_train
        )

[LibSVM]

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf',
                 SVC(C=0.25, coef0=1.0, degree=2, gamma=1.0, kernel='poly',
                     verbose=True))])

In [17]:
svm_predict = svm.predict(test_set[feature_names])
y_test = np.where(np.array(test_set[predict]), -1.0, 1.0)
print(confusion_matrix(y_test, svm_predict))
print(classification_report(y_test, svm_predict))

[[   0  132]
 [   0 1868]]
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       132
         1.0       0.93      1.00      0.97      1868

    accuracy                           0.93      2000
   macro avg       0.47      0.50      0.48      2000
weighted avg       0.87      0.93      0.90      2000



c:\Users\E\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\E\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\E\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [119]:
#### use predict set {-1, 1}
#df_clean = pd.read_feather('clean_features_221105_trades_ddt.f')
predict = 'spreaddown50ms'
split  = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=3407)
n = 10000
#ind_select =  np.random.choice(np.arange(len(df_clean)),size=n)
ind_select = np.arange(n) # no randomization
try:
    df_combine.reset_index(inplace=True)
except ValueError:
    pass # already rest index
df_select = df_combine.iloc[ind_select,:]
for train_index, test_index in split.split(df_select, df_select[predict]):
    train_set = df_combine[:n].loc[train_index]
    test_set = df_combine[:n].loc[test_index]
y_train = np.where(np.array(train_set[predict]), -1.0, 1.0)
svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=2, C=0.25, gamma=1.0, coef0=1.0, verbose=True)) # adjusting gamma for increased smoothness - trying to avoid trivial prediction
])
feature_names = ['ask_price', 'bid_price', 'ask_volume', 'bid_volume', 
                   'mid_price', 'spread', 
                   'dPask_dt', 'dPbid_dt', 'dVbid_dt', 'dVask_dt',
                   'avg_trade_price_10ms','avg_trade_volume_10ms',
                   'avg_trade_price_100ms','avg_trade_volume_100ms',
                   'avg_trade_price_1s','avg_trade_volume_1s',
                   'avg_trade_price_10s','avg_trade_volume_10s' 
                  ]
svm.fit(train_set[feature_names], 
        y_train
        )

[LibSVM]

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf',
                 SVC(C=0.25, coef0=1.0, degree=2, gamma=1.0, kernel='poly',
                     verbose=True))])

In [120]:
svm_predict = svm.predict(test_set[feature_names])
y_test = np.where(np.array(test_set[predict]), -1.0, 1.0)
print(confusion_matrix(y_test, svm_predict))
print(classification_report(y_test, svm_predict))

[[   1  728]
 [   2 1269]]
              precision    recall  f1-score   support

        -1.0       0.33      0.00      0.00       729
         1.0       0.64      1.00      0.78      1271

    accuracy                           0.64      2000
   macro avg       0.48      0.50      0.39      2000
weighted avg       0.53      0.64      0.49      2000



### 10 ms ahead prediction

In [29]:
#### use predict set {-1, 1}
#df_clean = pd.read_feather('clean_features_221105_trades_ddt.f')
df_combine = pd.read_feather('clean_combine_10e_10ms_AAPL_20200106.f')
predict = 'spreadup10ms'
split  = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=3407)
n = 10000
#ind_select =  np.random.choice(np.arange(len(df_clean)),size=n)
ind_select = np.arange(n) # no randomization
try:
    df_combine.reset_index(inplace=True)
except ValueError:
    pass # already rest index
df_select = df_combine.iloc[ind_select,:]
for train_index, test_index in split.split(df_select, df_select[predict]):
    train_set = df_combine[:n].loc[train_index]
    test_set = df_combine[:n].loc[test_index]
y_train = np.where(np.array(train_set[predict]), -1.0, 1.0)
svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=2, C=0.25, gamma=1.0, coef0=1.0, verbose=True)) # adjusting gamma for increased smoothness - trying to avoid trivial prediction
])
feature_names = ['ask_price', 'bid_price', 'ask_volume', 'bid_volume', 
                   'mid_price', 'spread', 
                   'dPask_dt', 'dPbid_dt', 'dVbid_dt', 'dVask_dt',
                   'avg_trade_price_10ms','avg_trade_volume_10ms',
                   'avg_trade_price_100ms','avg_trade_volume_100ms',
                   'avg_trade_price_1s','avg_trade_volume_1s',
                   'avg_trade_price_10s','avg_trade_volume_10s' 
                  ]
svm.fit(train_set[feature_names], 
        y_train
        )
svm_predict = svm.predict(test_set[feature_names])
y_test = np.where(np.array(test_set[predict]), -1.0, 1.0)
print(confusion_matrix(y_test, svm_predict))
print(classification_report(y_test, svm_predict))

[LibSVM][[   3  284]
 [   2 1711]]
              precision    recall  f1-score   support

        -1.0       0.60      0.01      0.02       287
         1.0       0.86      1.00      0.92      1713

    accuracy                           0.86      2000
   macro avg       0.73      0.50      0.47      2000
weighted avg       0.82      0.86      0.79      2000



In [130]:
svm_predict = svm.predict(test_set[feature_names])
y_test = np.where(np.array(test_set[predict]), -1.0, 1.0)
print(confusion_matrix(y_test, svm_predict))
print(classification_report(y_test, svm_predict))

[[   0  233]
 [   2 1765]]
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       233
         1.0       0.88      1.00      0.94      1767

    accuracy                           0.88      2000
   macro avg       0.44      0.50      0.47      2000
weighted avg       0.78      0.88      0.83      2000

